<a href="https://colab.research.google.com/github/AndresMontesDeOca/Laboratorio3/blob/main/Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Experiments

## Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
# from neuralprophet import NeuralProphet
import tensorflow as tf


import warnings
# warnings.filterwarnings('ignore', category=ValueWarning)
warnings.filterwarnings('ignore')

# Ajustar la opción para mostrar más filas
# pd.set_option('display.max_rows', None)

# Si también quieres mostrar más columnas
# pd.set_option('display.max_columns', None)


# Vamos a suprimir la notacion cientifica
pd.set_option("display.float_format", lambda x:"%.2f" %x)


ModuleNotFoundError: No module named 'neuralprophet'

## Carga Datos

In [2]:
# Code to read csv file into Colaboratory:
# !pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

################################# Datasets ###################################
# # Ventas
id = "158aOjqxaNO8l97yA6VWJkek_15YVLMhs"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('sell-in.txt')
data_ventas = pd.read_csv("sell-in.txt", sep="\t")
data_ventas['periodo'] = pd.to_datetime(data_ventas['periodo'], format='%Y%m')
data = data_ventas.copy()

# # Productos
id = "15JS_k86LS0sgJXma7BOVXWlyNcMwxdhE"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('tb_productos.txt')
data_productos = pd.read_csv("tb_productos.txt", sep="\t")

# # Stocks
id = "15EV-8f_U7onpA1AcTxxXeD-z8yVR4fQu"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('tb_stocks.txt')
data_stocks = pd.read_csv("tb_stocks.txt", sep="\t")
data_stocks['periodo'] = pd.to_datetime(data_stocks['periodo'], format='%Y%m')

# # Productos a predecir
id = "15LjADctFVwjzQFJvfJGFTEdgZx9xCoId"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('productos_a_predecir.txt')
data_productos_a_predecir = pd.read_csv("productos_a_predecir.txt", sep="\t")



## Agrupamiento Datos

In [6]:
# Filtrar el DataFrame 'data' para que solo contenga los 'product_id' presentes en 'data_productos_a_predecir'
data_filtered = data[data['product_id'].isin(data_productos_a_predecir['product_id'])]


# Agrupa los datos por 'product_id' y 'periodo', y calcula la suma de 'tn'
grouped_data = data_filtered.groupby(['product_id', 'periodo']).sum().reset_index()

# Crea un DataFrame pivoteado donde las filas son las fechas y las columnas son los product_id
pivot_data = grouped_data.pivot(index='periodo', columns='product_id', values='tn')

# Rellena los NaN
pivot_data = pivot_data.fillna(0)

# Asegúrate de que los nombres de las columnas sean strings
pivot_data.columns = pivot_data.columns.astype(str)

# Restablece el índice para asegurarse de que 'product_id' no sea un índice compuesto
pivot_data.columns.name = None

data_2019 = pivot_data.loc['2019']

In [10]:
data_2019

,20001,20002,20003,20004,20005,20006,20007,20008,20009,20010,...,21248,21252,21256,21259,21262,21263,21265,21266,21267,21276
periodo,,,,,,,,,,,,,,,,,,,,,
2019-01-01,1275.77351,1266.78751,964.76919,511.33713,363.58438,578.74461,377.66902,543.27828,465.47521,370.75591,...,0.02962,0.00000,0.03811,0.04234,0.02680,0.04376,0.00000,0.00000,0.00000,0.00000
2019-02-01,1259.09363,1043.01349,758.32657,441.70332,409.89950,479.99914,368.79546,476.98787,366.72969,337.76009,...,0.06209,0.00000,0.05081,0.05080,0.05786,0.05927,0.00000,0.00000,0.00000,0.00000
2019-03-01,1470.65653,1083.62552,638.04010,619.77084,488.21387,502.43741,576.23305,454.57037,525.47182,522.87583,...,0.05365,0.38391,0.05930,0.06777,0.06353,0.06636,0.01593,0.01707,0.21578,0.12249
2019-04-01,1647.63848,1287.62346,565.33774,466.70901,624.99880,835.47883,511.54995,403.69191,391.28033,446.72413,...,0.03106,0.23471,0.01835,0.02117,0.01412,0.01835,0.36405,0.36178,0.12291,0.10173
2019-05-01,1629.78233,1034.98927,590.12515,603.31081,897.26297,527.68846,394.67651,486.36682,578.48564,408.41680,...,0.01553,0.47194,0.01553,0.01976,0.01130,0.02258,0.17635,0.17634,0.24451,0.09283
2019-06-01,1109.93769,928.36431,662.38654,667.19411,876.39696,458.04180,536.13689,567.42091,610.39590,600.25060,...,0.00988,0.07824,0.00988,0.00988,0.01270,0.00988,0.05121,0.05235,0.05882,0.04086
2019-07-01,1678.99318,1066.44999,715.20314,521.71519,745.74978,343.11053,573.37257,524.04994,716.07987,463.91662,...,0.02116,0.02691,0.02822,0.04657,0.03953,0.03387,0.02959,0.05916,0.07452,0.00223
2019-08-01,1261.34529,813.78215,635.59563,482.13372,536.66800,262.73593,307.82899,233.00983,520.41758,199.86233,...,0.02117,0.13446,0.02965,0.03106,0.02258,0.03388,0.01593,0.01480,0.04054,0.01265
2019-09-01,1660.00561,1090.18771,967.77116,786.17140,879.52808,409.95501,369.74894,330.56343,558.45719,524.94628,...,0.01411,0.14427,0.00423,0.00564,0.01552,0.01128,0.01707,0.02844,0.01830,0.01856


## Normalizar Data

In [11]:
import pandas as pd

def normalize_series(df):
    """
    Normaliza cada serie de tiempo (columna) de manera individual.

    Args:
        df (pd.DataFrame): DataFrame con series de tiempo de distintos productos, cada columna es un producto.

    Returns:
        normalized_df (pd.DataFrame): DataFrame con las series normalizadas.
        min_max_values (dict): Diccionario con los valores min y max de cada columna.
    """
    min_max_values = {}
    normalized_df = pd.DataFrame()

    for column in df.columns:
        min_value = df[column].min()
        max_value = df[column].max()
        min_max_values[column] = (min_value, max_value)

        normalized_df[column] = (df[column] - min_value) / (max_value - min_value)

    return normalized_df, min_max_values


In [14]:
data, min_max_data = normalize_series(data_2019)

<ipython-input-11-5994cf48ae33>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  normalized_df[column] = (df[column] - min_value) / (max_value - min_value)
<ipython-input-11-5994cf48ae33>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  normalized_df[column] = (df[column] - min_value) / (max_value - min_value)
<ipython-input-11-5994cf48ae33>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once 

In [16]:
data['20001'].describe()

count    12.000000
mean      0.605908
std       0.336903
min       0.000000
25%       0.285084
50%       0.663793
75%       0.921367
max       1.000000
Name: 20001, dtype: float64

In [17]:
data

,20001,20002,20003,20004,20005,20006,20007,20008,20009,20010,...,21248,21252,21256,21259,21262,21263,21265,21266,21267,21276
periodo,,,,,,,,,,,,,,,,,,,,,
2019-01-01,0.291423,0.388594,0.774049,0.111773,0.000000,0.551746,0.260205,0.935110,0.282655,0.426820,...,0.378088,0.000000,0.615217,0.590697,0.296764,0.599858,0.000000,0.000000,0.000000,0.000000
2019-02-01,0.262111,0.196638,0.373989,0.000000,0.073145,0.379338,0.227144,0.756935,0.000000,0.344410,...,1.000000,0.000000,0.845833,0.726863,0.891442,0.874469,0.000000,0.000000,0.000000,0.000000
2019-03-01,0.633890,0.231475,0.140888,0.285826,0.196825,0.418515,1.000000,0.696681,0.454393,0.806751,...,0.838345,0.813472,1.000000,1.000000,1.000000,1.000000,0.043758,0.047183,0.882500,1.000000
2019-04-01,0.944900,0.406468,0.000000,0.040138,0.412848,1.000000,0.759008,0.559930,0.070275,0.616556,...,0.405669,0.497330,0.256401,0.249960,0.053992,0.149965,1.000000,1.000000,0.502679,0.830517
2019-05-01,0.913522,0.189755,0.048035,0.259405,0.842830,0.462603,0.323570,0.782143,0.606142,0.520881,...,0.108217,1.000000,0.205193,0.227265,0.000000,0.224858,0.484411,0.487423,1.000000,0.757858
2019-06-01,0.000000,0.098290,0.188069,0.361948,0.809877,0.341001,0.850613,1.000000,0.697484,1.000000,...,0.000000,0.165784,0.102597,0.068244,0.026805,0.000000,0.140667,0.144701,0.240563,0.333578
2019-07-01,1.000000,0.216742,0.290421,0.128431,0.603548,0.140333,0.989343,0.883428,1.000000,0.659496,...,0.216051,0.057020,0.435627,0.658780,0.540494,0.424752,0.081280,0.163525,0.304773,0.018206
2019-08-01,0.266068,0.000000,0.136151,0.064897,0.273348,0.000000,0.000000,0.101172,0.439925,0.000000,...,0.216242,0.284909,0.461594,0.409142,0.215968,0.424929,0.043758,0.040909,0.165801,0.103274
2019-09-01,0.966633,0.237104,0.779866,0.552924,0.814822,0.257042,0.230697,0.363376,0.548812,0.811922,...,0.081019,0.305696,0.000000,0.000000,0.080796,0.024788,0.046889,0.078611,0.074844,0.151523
